# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- RN - running
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [1]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER, DESIRED_MEAN_VALUE

rep_data, wide_use_data, feedback_data = prepare_data(PATH, DESIRED_MEAN_VALUE)
month, mean, std, df = wide_use_data
df

ValueError: cannot convert float NaN to integer

In [ ]:
rep_data

([0, 61, 29, 90, 244, 6, '4, 5, 9, 11, 15, 19'],
 [['OD', 'LE', 'RN', 'GM'],
  16,
  [45.48, 4.0, 2.08, 24.9],
  [21, 12, 6, 15],
  {1: 1, 2: 3, 3: 1, 4: 10, 5: 2}])

# Working with 'hobby' data

Get the file content

In [7]:
import re
import os

import pandas as pd


# PATH = "test.txt"
PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/2023 - What I read, watched and played.txt"
PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/2022 - What I read, watched and played.txt"
PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/2021 - What I read, watched and played.txt"
# Read the text file with UTF-8 encoding
with open(PATH, "r", encoding="utf-8") as file:
    content = file.read()
print(content)

Books:
1. "The Prince" by Nicolo Machiavelli on Jan 4th, 2021
2. "Descriptions of the methods adopted by the..." by Nicolo Machiavelli on Jan 4th, 2021
3. "The life of Castruccio Castracani of Lucca" by Nicolo Machiavelli on Jan 4th, 2021
4. "Alice's Adventures in Wonderland" by Lewis Carroll on Jan 10th, 2021
5. "Statistics Done Wrong" by Alex Reinhart on Jan 13th, 2021
6. "The Wonderful Wizard of Oz" by L. Frank Baum on Jan 14th, 2021
7. "Розмисли на одинці з собою" by Marcus Aurelius on Jan 17th, 2021 
8. "Harry Potter and The Philosopher's Stone" by J.K. Rowling | audiobook on Jan 17th, 2021
9. "Harry Potter and the Chamber of Secrets" by J.K. Rowling | audiobook on Jan 20th, 2021
10. "Harry Potter and the Prisoner of Azkaban" by J.K. Rowling | audiobook on Jan 22nd, 2021
11. "Essays on the Theory of Numbers" by Richard Dedekind on Jan 26th, 2021
12. "Harry Potter and the Goblet of Fire" by J.K. Rowling | audiobook on Jan 27th, 2021
13. "Harry Potter and the Order of the Phoenix" b

Get movie df

In [8]:
# Find the 'Movies:' section using the regular expression
movies_section = (
    re.search(r"Movies:(.*?)(?=\n\n|$)", content, re.DOTALL).group(1).strip()
)
# Define a regular expression pattern for extracting movie information
movies_pattern = re.compile(
    r"\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4})"
)
# Extract movie information using the pattern
movies_data = movies_pattern.findall(movies_section)
cols = ["Name", "Month", "Day", "Year"]
# Create a pandas DataFrame for movies
movies_df = pd.DataFrame(movies_data, columns=cols)
# Shorten the month to the first three letters
movies_df["Month"] = movies_df["Month"].apply(lambda month: month[:3])
movies_df

,Name,Month,Day,Year
0,Harry Potter and the Philosopher's Stone,Jan,4,2021
1,Harry Potter and the Chamber of Secrets,Jan,5,2021
2,Harry Potter and the Prisoner of Azkaban,Jan,6,2021
3,Harry Potter and the Goblet of Fire,Jan,6,2021
4,Harry Potter and the Order of the Phoenix,Jan,7,2021
...,...,...,...,...
106,Commando,Dec,22,2021
107,Groundhog Day,Dec,25,2021
108,Scarface,Dec,26,2021
109,Point Break,Dec,27,2021


Get games df

In [9]:
# Find the 'Games:' section using the regular expression
games_section = re.search(r"Games:(.*?)$", content, re.DOTALL).group(1).strip()
# Define a regular expression pattern for extracting game information
games_pattern = re.compile(
    r"\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4}) \((\d+) hours\)"
)
# Extract game information using the pattern
games_data = games_pattern.findall(games_section)
# 'PTH' stands for 'Play Time in Hours'a
cols = ["Name", "Month", "Day", "Year", "PTH"]
# Create a pandas DataFrame for games
games_df = pd.DataFrame(games_data, columns=cols)
# Shorten the month to the first three letters
games_df["Month"] = games_df["Month"].apply(lambda month: month[:3])
games_df

,Name,Month,Day,Year,PTH
0,Kingdom New Lands,Dec,29,2021,14


Get books df

In [10]:
def get_books_df(content):
    """
    Summary:
        Extracts book information from a provided text content and creates a DataFrame.
    Args:
        content (str): The text content containing book information.
    Returns:
        pandas.DataFrame: A DataFrame containing book details with columns:
            'Name', 'Author', 'Audiobook', 'Month', 'Day', and 'Year'.
    What function does in more detail:
        The function extracts book information from the provided content, including
        book titles, authors, audiobook status, and completion date. It parses the
        content using regular expressions and processes each line to create a structured
        DataFrame for further analysis.

        Columns of the df and what they mean:
        - 'Name': The title of the book
        - 'Author': The author's name
        - 'Audiobook': A boolean indicating whether it's an audiobook (True) or not (False)
        - 'Month': The completion month (abbreviated to the first three letters)
        - 'Day': The completion day
        - 'Year': The completion year
        The last three cols ('Month', 'Day', and 'Year') show when you finished reading the book
    """
    # Find the 'Books:' section using string manipulation
    books_section = (
        re.search(r"Books:(.*?)(?=Movies:|Games:)", content, re.DOTALL).group(1).strip()
    )
    # Split the section into lines
    books_lines = books_section.split("\n")
    # Remove all data before the first double quote in each line
    books_lines = [line.split('"', 1)[1] for line in books_lines]
    # Initialize variables to store book data
    books_data = []
    audiobook_marker = "| audiobook"
    # Process each line in the book section
    for line in books_lines:
        cur_book = {}
        # Audiobook entry
        if audiobook_marker in line:
            # Split the data on book data (book name and author)
            # and date (Month, day, and year)
            name_author, date = line.split(audiobook_marker)
            name, author = name_author.split('" by ')
            # Use 'strip()' to remove trailing whitespaces
            cur_book["Name"] = name.strip()
            cur_book["Author"] = author.strip()
            cur_book["Audiobook"] = True
        # Regular book entry
        else:
            # Handle the case when there's ' on ' in book name
            # e.g., 'Essays on the Theory of Numbers'
            try:
                name_author_pair, date = line.split(" on ")
            except ValueError:
                line_list = line.split(" on ")
                name_author_pair = " on ".join(line_list[:-1])
                date = line_list[-1]
            name, author = name_author_pair.split('" by ')
            # Use 'strip()' to remove trailing whitespaces
            cur_book["Name"] = name.strip()
            cur_book["Author"] = author.strip()
            cur_book["Audiobook"] = False
        # Remove trailing whitespace before the string
        # Otherwise, regex won't work
        date = date.strip()
        # Extract date info using regex
        match = re.match(
            r"(?:on )?([A-Za-z]+) (\d{1,2})(?:st|nd|rd|th)?, (\d{4})", date
        )
        cur_book["Month"] = match.group(1)[:3]
        cur_book["Day"] = int(match.group(2))
        cur_book["Year"] = int(match.group(3))
        books_data.append(cur_book)
    # Create a pandas DataFrame for books and return it
    return pd.DataFrame(
        books_data, columns=["Name", "Author", "Audiobook", "Month", "Day", "Year"]
    )


books_df = get_books_df(content)
books_df.head(50)

,Name,Author,Audiobook,Month,Day,Year
0,The Prince,Nicolo Machiavelli,False,Jan,4,2021
1,Descriptions of the methods adopted by the...,Nicolo Machiavelli,False,Jan,4,2021
2,The life of Castruccio Castracani of Lucca,Nicolo Machiavelli,False,Jan,4,2021
3,Alice's Adventures in Wonderland,Lewis Carroll,False,Jan,10,2021
4,Statistics Done Wrong,Alex Reinhart,False,Jan,13,2021
5,The Wonderful Wizard of Oz,L. Frank Baum,False,Jan,14,2021
6,Розмисли на одинці з собою,Marcus Aurelius,False,Jan,17,2021
7,Harry Potter and The Philosopher's Stone,J.K. Rowling,True,Jan,17,2021
8,Harry Potter and the Chamber of Secrets,J.K. Rowling,True,Jan,20,2021
9,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,True,Jan,22,2021


Work with all the files in the folder to get book data from all recorded years

In [12]:
def process_files_in_folder(folder_path):
    # Create an empty list to store DataFrames from each file
    data_frames = []
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            # Read the content of the file
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
            # Apply the get_books_df function to the file's content
            books_df = get_books_df(content)
            # Append the resulting DataFrame to the list
            data_frames.append(books_df)
    # Concatenate all DataFrames into a single DataFrame and return it
    return pd.concat(data_frames, ignore_index=True)


FOLDER_PATH = "C:/Users/San/Documents/inf/time monitoring/book, movies, and other stuff I completed by year/"
all_books_df = process_files_in_folder(FOLDER_PATH)

In [13]:
all_books_df

,Name,Author,Audiobook,Month,Day,Year
0,The Prince,Nicolo Machiavelli,False,Jan,4,2021
1,Descriptions of the methods adopted by the...,Nicolo Machiavelli,False,Jan,4,2021
2,The life of Castruccio Castracani of Lucca,Nicolo Machiavelli,False,Jan,4,2021
3,Alice's Adventures in Wonderland,Lewis Carroll,False,Jan,10,2021
4,Statistics Done Wrong,Alex Reinhart,False,Jan,13,2021
...,...,...,...,...,...,...
300,The Blue Carbuncle,Arthur Conan Doyle,True,Dec,4,2023
301,Огненне коло. Повість про трагедію під Бродами,Іван Багряний,False,Dec,7,2023
302,A Christmas Carol,Charles Dickens,True,Dec,18,2023
303,Лісова Пісня,Леся Українка,False,Dec,19,2023


In [17]:
all_books_df.to_csv("books_data.csv")

In [15]:
all_books_df["Author"].value_counts()

Joe Abercrombie                           33
Robert E. Howard                          18
Seneca                                    11
Roald Dahl                                10
J.R.R. Tolkien                             9
                                          ..
Kahlil Gibran                              1
Andriy Burkov                              1
C. S. Lewis | The Chronicles of Narnia     1
Howard Pyle                                1
Леся Українка                              1
Name: Author, Length: 137, dtype: int64

In [16]:
all_books_df["Audiobook"].value_counts()

True     251
False     54
Name: Audiobook, dtype: int64